<a href="https://colab.research.google.com/github/Camilojaravila/202120_ISIS4219_01-MACHINE_LEARNING_TECHNIQUES/blob/main/Lab_3/Taller/Laboratorio_III_Taller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://bloqueneon.uniandes.edu.co/content/enforced/52233-202120_ISIS4219_01/Laboratorios.png?_&d2lSessionVal=guLBE9l4cI9Tip6PM5JSzKgp6&_&d2lSessionVal=iDtWjzbHb92iC3sVmHUqajJ2k&_&d2lSessionVal=g2xptlvZ5GrV3W5yGlhIgeBnN" ><br>
# Machine Learning Techniques - ISIS4219

Segundo Semestre - 2021



## Integrantes

*   Daniel Mateo Guatibonza Solano - 201611360
*   Raquel Andrea Cerda Montañez - 201533520
*   Sergio Eduardo Cárdenas Landazabal - 201613444



## **Problema**

Ante la situación actual de sanidad pública en el mundo a causa del COVID-19. La detección temprana de esta enfermedad en los pacientes, la automatización de procesos de diagnóstico y el apoyo a los diferentes expertos de esta enfermedad juega un papel muy importante para enfrentar una pandemia el cual ha registrado más de 230 millones de contagios, dejando sin vida a 4 millones de personas alrededor del mundo (OMS, 2021).

Sin embargo, todo este tiempo de cuarentena no solo ha servido para frenar la tasa de contagio del virus. También nos ha permitido recolectar diferentes radiografias de pacientes quienes han sufrido esta enfermedad, junto a otras enfermedades que pueden llegar a tener una confusión con la enfermedad provocada por el virus. Por ello, es importante determinar con exactitud el tipo de enfermedad para poder recomendar el mejor tratamiento para el paciente.

Se han recolectado alrededor de 21 mil imágenes provenientes de diferentes paises del medio oriente como Catar, Pakistan y Malasia. Y, ante la reactivación económica que se quiere llevar a cabo, se teme una nueva ola de contagios, por lo que poder identificar estos casos dadas sus radiografías autmáticamente, sería de gran ayuda para los trabajadores de la Salud para determinar el tratamiento a los pacientes.

Para eso, le han pedido que construya un modelo capaz de recibir esas imágenes y determinar bajo que probabilidad puede pertenecer a cada una de las clases de las enfermedades proporcionadas. Y así mismo determinar cuál sería el diagnóstico de ese paciente. Como sugerencia, le han pedido construir una Red Neuronal Artificial (ANN) la cual permita realizar este proceso de manera automática y sin supervisión de los expertos en el tema. 

Finalmente, ante la urgencia del problema, le han pedido que utilicen las imágenes en escala de grises para poder realizar una carga y un almacenamiento de manera más óptima, y utilizar directamente como entrada esta imagen.

**Fuente de Datos:** https://www.kaggle.com/preetviradiya/covid19-radiography-dataset

**Referencias:**

Coronavirus disease (COVID-19) – World Health Organization. (2021). Retrieved 5 October 2021, from https://www.who.int/emergencies/diseases/novel-coronavirus-2019?gclid=Cj0KCQjwwY-LBhD6ARIsACvT72PqduqyNWdB9YQ3awfqyC9FZ1y5Zy7srhp8wrVgEPPmzW4Jgll_Di8aAiDuEALw_wcB


## Librerías necesarias
En primer lugar, se importan todas las librerías requeridas para realizar la lectura, preprocesamiento y construcción de los modelos incluyendo las librerías requeridas para la selección de hiperparámetros.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print('Tensorflow version:', tf.__version__)

Tensorflow version: 2.8.0-dev20211020


## Lectura de datos
Considerando que la entrada de los modelos aa 

In [2]:
batch_size = 32
img_height = 299
img_width = 299

data_dir = './COVID-19_Radiography_Dataset'

train_ds = tf.keras.utils.image_dataset_from_directory(
                    data_dir,
                    validation_split=0.2,
                    subset="training",
                    color_mode='grayscale',
                    label_mode='categorical',
                    seed=0,
                    image_size=(img_height, img_width),
                    batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(
                    data_dir,
                    validation_split=0.2,
                    subset="validation",
                    color_mode='grayscale',
                    label_mode='categorical',
                    seed=0,
                    image_size=(img_height, img_width),
                    batch_size=batch_size)
class_names = train_ds.class_names
print('Clases:', class_names)

Found 21165 files belonging to 4 classes.
Using 16932 files for training.
Found 21165 files belonging to 4 classes.
Using 4233 files for validation.
Clases: ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']


In [3]:
def contar_por_clase(ds):
    cant_covid = 0
    cant_normal = 0
    cant_lung_opacity = 0
    cant_viral_pneumonia = 0
    for ruta in ds.file_paths:
        if 'Normal' in ruta:
            cant_normal += 1
        elif 'Lung_Opacity' in ruta:
            cant_lung_opacity += 1
        elif 'Viral Pneumonia' in ruta:
            cant_viral_pneumonia += 1
        elif 'COVID' in ruta:
            cant_covid += 1
    print('COVID:', cant_covid)
    print('Normal:', cant_normal)
    print('Lung Opacity:', cant_lung_opacity)
    print('Viral Pneumonia:', cant_viral_pneumonia)

print('------------------------------------------------------------')
print('Cantidad de datos por clase en el conjunto de entrenamiento:')
contar_por_clase(train_ds)
print('------------------------------------------------------------')
print('Cantidad de datos por clase en el conjunto de prueba:')
contar_por_clase(test_ds)

------------------------------------------------------------
Cantidad de datos por clase en el conjunto de entrenamiento:
COVID: 2855
Normal: 8179
Lung Opacity: 4828
Viral Pneumonia: 1070
------------------------------------------------------------
Cantidad de datos por clase en el conjunto de prueba:
COVID: 761
Normal: 2013
Lung Opacity: 1184
Viral Pneumonia: 275


In [4]:
def dar_X_y(ds):
    X = []
    y = []
    for image_batch, labels_batch in train_ds:
        for i in range(image_batch.shape[0]):
            X.append(image_batch[i].numpy())
            y.append(labels_batch[i].numpy())
    return np.array(X), np.array(y)
X_train, y_train = dar_X_y(train_ds)
X_test, y_test = dar_X_y(test_ds)

In [5]:
model = Sequential()
model.add(Flatten(input_shape = (img_height, img_width)))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 89401)             0         
                                                                 
 dense (Dense)               (None, 512)               45773824  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
 activation_1 (Activation)   (None, 4)                 0         
                                                                 
Total params: 45,775,876
Trainable params: 45,775,876
No

In [6]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callbacks = [early_stopping]

In [8]:
history = model.fit(X_train, y_train, epochs=100, callbacks=callbacks)

Epoch 1/100
530/530 [==============================] - 203s 165ms/step - loss: 574.2537 - accuracy: 0.4716
Epoch 2/100
530/530 [==============================] - 84s 159ms/step - loss: 3.9601 - accuracy: 0.4833
Epoch 3/100
530/530 [==============================] - 84s 158ms/step - loss: 1.2035 - accuracy: 0.4831
Epoch 4/100
530/530 [==============================] - 85s 161ms/step - loss: 1.1866 - accuracy: 0.4832
Epoch 5/100
530/530 [==============================] - 85s 160ms/step - loss: 1.1847 - accuracy: 0.4830
Epoch 6/100
530/530 [==============================] - 83s 157ms/step - loss: 1.1841 - accuracy: 0.4831
Epoch 7/100
530/530 [==============================] - 84s 158ms/step - loss: 1.1840 - accuracy: 0.4831
Epoch 8/100
530/530 [==============================] - 84s 158ms/step - loss: 1.1839 - accuracy: 0.4831
Epoch 9/100
530/530 [==============================] - 84s 158ms/step - loss: 1.1839 - accuracy: 0.4832
Epoch 10/100
530/530 [==============================] - 84s 1

In [9]:
def entrenar_red(nn1 = 512, nn2 = 100, nn3 = 25, n_layers= 3, dropout = 0.1, activacion_oculta = 'relu'):
    clf = Sequential(name='Mi_Red')
    clf.add(Flatten(input_shape=(img_height, img_width)))

    output = 4
    first = True
    
    num_neuronas = [nn1, nn2, nn3]
    for i in range(n_layers):
        clf.add(Dense(num_neuronas[i], activation=activacion_oculta, name='Capa_{}'.format(i)))
        clf.add(Dropout(dropout, name='Dropout_{}'.format(i)))
        
    clf.add(Dense(output, activation='softmax', name= 'Capa_Salida'))
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return clf

# Modelo que utiliza el GridSearch
modelCV = KerasClassifier(build_fn=entrenar_red, epochs=100, verbose=1, shuffle=True)

In [10]:
pipe = Pipeline([('red_neuronal', modelCV)])

param_grid = dict(red_neuronal__nn1 = [512, 2048, 8192],
                  red_neuronal__nn2 = [128, 512, 2048],
                  red_neuronal__nn3 = [32, 128, 512],
                  red_neuronal__n_layers = [1, 2, 3],
                  red_neuronal__dropout = [0.1, 0.2, 0.3],
                  red_neuronal__activacion_oculta = ['relu','sigmoid'])

def my_custom_acc(y_true, y_pred):
    return accuracy_score(y_true.argmax(axis=-1), y_pred)

score = make_scorer(my_custom_acc, greater_is_better=True)
grid = RandomizedSearchCV(pipe, param_grid, scoring=score, verbose=1, cv=3, n_iter=25, random_state=0)

In [11]:
grid.fit(X_train, y_train, red_neuronal__callbacks=callbacks)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Epoch 1/100
353/353 [==============================] - ETA: 0s - loss: 3031.2583 - accuracy: 0.502 - 1926s 2s/step - loss: 3031.2583 - accuracy: 0.5022
Epoch 2/100
353/353 [==============================] - 551s 2s/step - loss: 7.7316 - accuracy: 0.4826
Epoch 3/100
353/353 [==============================] - 509s 1s/step - loss: 1.2975 - accuracy: 0.4856
Epoch 4/100
353/353 [==============================] - 334s 946ms/step - loss: 2.3425 - accuracy: 0.4860
Epoch 5/100
353/353 [==============================] - 228s 645ms/step - loss: 1.1614 - accuracy: 0.4863
Epoch 6/100
353/353 [==============================] - 223s 632ms/step - loss: 1.1811 - accuracy: 0.4862
Epoch 7/100
353/353 [==============================] - 224s 636ms/step - loss: 1.1770 - accuracy: 0.4864
Epoch 8/100
353/353 [==============================] - 223s 631ms/step - loss: 1.1803 - accuracy: 0.4861
Epoch 00008: early stopping
Epoch 1/100
353/353 [=========

MemoryError: Unable to allocate 1.88 GiB for an array with shape (5644, 299, 299, 1) and data type float32

In [ ]:
best_model = grid.best_estimator_
print(best_model.get_params())
best_model['red_neuronal'].model.summary()

In [ ]:
score = model.evaluate(X_train, y_train, verbose=1)
print('Train loss: {:4f}\nTrain Accuracy: {:4f}'.format(score[0], score[1]))